In [2]:
# TensorFlow 및 관련 라이브러리 임포트
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv2D, MaxPooling2D, Flatten, Dropout, Embedding, Bidirectional, GRU
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy, CategoricalCrossentropy
from tensorflow.keras.metrics import Accuracy, Precision, Recall

# 자연어처리를 위한 NLTK 라이브러리 임포트
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer

# 자연어처리를 위한 SpaCy 라이브러리 임포트
import spacy
from spacy.tokens import Doc, Span, Token
# 모델 다운로드
!python -m spacy download en_core_web_sm
nlp = spacy.load('en_core_web_sm')

# 자연어처리를 위한 Gensim 라이브러리 임포트
import gensim
from gensim.models import Word2Vec, FastText
from gensim.corpora.dictionary import Dictionary
from gensim.models.phrases import Phrases, Phraser

# 기타 유용한 라이브러리 임포트
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
from collections import Counter
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score, roc_auc_score, precision_score, recall_score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kosmo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kosmo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\kosmo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --------------------------------------- 0.0/12.8 MB 660.6 kB/s eta 0:00:20
     --------------------------------------- 0.0/12.8 MB 326.8 kB/s eta 0:00:40
     --------------------------------------- 0.1/12.8 MB 508.4 kB/s eta 0:00:26
     -------- ------------------------------- 2.7/12.8 MB 14.1 MB/s eta 0:00:01
     -------------------------- ------------- 8.4/12.8 MB 33.4 MB/s eta 0:00:01
     --------------------------------------  12.8/12.8 MB 93.9 MB/s eta 0:00:01
     --------------------------------------  12.8/12.8 MB 93.9 MB/s eta 0:00:01
     --------------------------------------- 12.8/12.8 MB 73.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
import_

In [3]:
import warnings
warnings.filterwarnings(action='ignore')

import  

In [1]:
import_tf

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kosmo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kosmo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\kosmo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --------------------------------------- 0.0/12.8 MB 330.3 kB/s eta 0:00:39
     --------------------------------------- 0.0/12.8 MB 262.6 kB/s eta 0:00:49
     --------------------------------------- 0.1/12.8 MB 409.6 kB/s eta 0:00:32
     - -------------------------------------- 0.5/12.8 MB 2.8 MB/s eta 0:00:05
     ---------- ----------------------------- 3.2/12.8 MB 14.7 MB/s eta 0:00:01
     ------------------ --------------------- 5.9/12.8 MB 22.1 MB/s eta 0:00:01
     ------------------------- -------------- 8.3/12.8 MB 27.9 MB/s eta 0:00:01
     ------------------------------ -------- 10.0/12.8 MB 30.6 MB/s eta 0:00:01
     ----------------------------------- --- 11.7/12.8 MB 54.4 MB/s eta 0:00:01
     --------------------------------------  12.8/12.8 MB 59.5 MB/s eta 0:00:01
     --------------------------------------- 12.8/12.8 MB 43.5 MB/s eta 0:00:00
✔ Download and installation successful
You can no

In [10]:
nsmc_train_df = pd.read_csv('./nsmc/ratings_train.txt', encoding='utf8', sep='\t', engine='python')
nsmc_train_df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [13]:
nsmc_train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [14]:
nsmc_train_df = nsmc_train_df[nsmc_train_df['document'].notnull()]

In [15]:
nsmc_train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 149995 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        149995 non-null  int64 
 1   document  149995 non-null  object
 2   label     149995 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.6+ MB


In [18]:
nsmc_train_df['label'].value_counts()

label
0    75170
1    74825
Name: count, dtype: int64

In [19]:
nsmc_train_df['document'] = nsmc_train_df['document'].apply(lambda x:re.sub(r'[^ ㄱ-ㅣ가-힣]+',"",x)) 
nsmc_train_df.head() 

C:\Users\kosmo\AppData\Local\Temp\ipykernel_3952\4209811118.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nsmc_train_df['document'] = nsmc_train_df['document'].apply(lambda x:re.sub(r'[^ ㄱ-ㅣ가-힣]+',"",x))


,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...,1


In [22]:
nsmc_test_df = pd.read_csv('./nsmc/ratings_test.txt',encoding='utf8',sep='\t',engine='python')
nsmc_test_df.head()

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [23]:
nsmc_test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        50000 non-null  int64 
 1   document  49997 non-null  object
 2   label     50000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.1+ MB


In [24]:
nsmc_test_df = nsmc_test_df[nsmc_test_df['document'].notnull()]

In [25]:
nsmc_test_df['label'].value_counts()

label
1    25171
0    24826
Name: count, dtype: int64

In [26]:
nsmc_test_df['document'] = nsmc_test_df['document'].apply(lambda x: re.sub(r'[^ ㄱ-ㅣ가-힣]+',"",x))


In [27]:
from konlpy.tag import Okt

okt = Okt()

In [29]:
def okt_tokenizer(text):
    tokens = okt.morphs(text)
    return tokens

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(tokenizer = okt_tokenizer, ngram_range=(1,2), min_df=3, max_df=0.9, token_pattern=None)
tfidf.fit(nsmc_train_df['document'])
nsmc_train_tfidf = tfidf.transform(nsmc_train_df['document'])

In [31]:
from sklearn.linear_model import LogisticRegression

SA_lr = LogisticRegression(random_state = 0,  max_iter=500)
SA_lr.fit(nsmc_train_tfidf, nsmc_train_df['label'])

LogisticRegression(max_iter=500, random_state=0)